In [ ]:
!pip install pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
credits.sample(n=3)

,movie_id,title,cast,crew
2332,5279,Gosford Park,"[{""cast_id"": 2, ""character"": ""Mrs. Wilson"", ""c...","[{""credit_id"": ""52fe4401c3a36847f807ceab"", ""de..."
4505,375950,The Country Doctor,"[{""cast_id"": 1, ""character"": ""Jean-Pierre Wern...","[{""credit_id"": ""568ae5109251412e5202a3bf"", ""de..."
2677,62008,Good Deeds,"[{""cast_id"": 3, ""character"": ""Glen Deeds"", ""cr...","[{""credit_id"": ""52fe466ec3a368484e09057d"", ""de..."


In [ ]:
movies.shape

(4803, 20)

In [ ]:
movies.head()

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.shape

(4809, 23)

### non-numeric
budget
homepage
id
original_language
original_title
popularity
production_comapny
production_countries

------------------------
### numeric

popularity
vote_average
vote_count
release-date()

In [ ]:
movies.info()

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
def convert(text):
  L = []
  for i in ast.literal_eval(text):
    L.append(i['name'])
  return L

In [ ]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
def convert_cast(text):
  L = []
  counter = 0
  for i in ast.literal_eval(text):
    if(counter < 3):
      L.append(i['name'])
    counter += 1
  return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [ ]:
movies['cast'].apply(lambda x:x[0:3])

In [ ]:
def fetch_director(text):
  L = []
  counter = 0
  for i in ast.literal_eval(text):
    if(i['job'] == 'Director'):
      L.append(i['name'])
    counter += 1
  return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
def remove_space(text):
  L1= []
  for i in text:
    L1.append(i.replace(" ", ""))
  return L1

In [ ]:
#removing the spaces in text
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [ ]:
# splitting the overview
movies['overview'] = movies['overview'].apply(lambda a:a.split())

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies_new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [ ]:
movies_new.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [ ]:
#replace ',' with ' '
movies_new['tags'] = movies_new['tags'].apply(lambda x: " ".join(x))

In [ ]:
movies_new.sample(n=20)

In [ ]:
#problem we have alot of english stopping points that dont contripute to the meaning
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(movies_new['tags']).toarray()

In [ ]:
vector.shape

(4806, 5000)

In [ ]:
#applying similarity to the matrix
similarity = cosine_similarity(vector)

In [ ]:
similarity[0]

array([1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
       0.        ])

In [ ]:
#select movie by name
movies_new[movies_new['title'] == 'Avatar'].index[0]

0

In [ ]:
def recommend(title):
  title_index =  movies_new[movies['title'] == title].index[0]
  movies_sorted = sorted(list(enumerate(similarity[title_index])), reverse=True, key = lambda x: x[1])
  for i in movies_sorted[0:15]:
    print(movies_new.iloc[i[0]].title)
    print(i)

In [ ]:
recommend('Avatar')

Avatar
(0, 1.0)
Titan A.E.
(539, 0.26089696604360174)
Small Soldiers
(1194, 0.2581988897471611)
Ender's Game
(260, 0.25110592822973776)
Aliens vs Predator: Requiem
(1216, 0.24944382578492943)
Independence Day
(507, 0.24846467329894412)
Krull
(1444, 0.24397501823713333)
Lifeforce
(1920, 0.243599382882345)
Battle: Los Angeles
(582, 0.24147264420814754)
Falcon Rising
(3730, 0.23904572186687872)
Edge of Tomorrow
(74, 0.22677868380553634)
Apollo 18
(3608, 0.2238868314198225)
The Fifth Element
(322, 0.22230800575069137)
The Lovers
(83, 0.22190115272469205)
Independence Daysaster
(4192, 0.22131333406899523)


In [ ]:
recommend('The Fifth Element')

The Fifth Element
(322, 1.0)
Race to Witch Mountain
(931, 0.31068488300060004)
Heavy Metal
(3154, 0.2847473987257497)
Titan A.E.
(539, 0.22555354977384034)
Avatar
(0, 0.22230800575069137)
Jupiter Ascending
(61, 0.21029989116983447)
Starship Troopers
(300, 0.20795009796401454)
John Carter
(4, 0.2029385157727364)
Dragonball Evolution
(1658, 0.1934477640657483)
Krull
(1444, 0.18983159915049982)
All or Nothing
(3198, 0.18983159915049982)
The Helix... Loaded
(4405, 0.18983159915049982)
Star Trek: Insurrection
(581, 0.1898315991504998)
Impostor
(1275, 0.1898315991504998)
Mad Max: Fury Road
(127, 0.18857857160117852)
